In [16]:
import numpy as np
import astropy.io.fits as fits
import pandas as pd
import math
pd.set_option('display.max_colwidth', None)

# Useful
# http://ps1images.stsci.edu/ps1_dr2_api.html

from astropy.io import ascii
from astropy.table import Table

import sys
import re
import numpy as np
import matplotlib.pyplot as plt
import json
import requests

try: # Python 3.x
    from urllib.parse import quote as urlencode
    from urllib.request import urlretrieve
except ImportError:  # Python 2.x
    from urllib import pathname2url as urlencode
    from urllib import urlretrieve

try: # Python 3.x
    import http.client as httplib 
except ImportError:  # Python 2.x
    import httplib
    

# My function: get ps1_psc score from df
def get_psc_score(df):
    ps_score_l = []
    for i, (ztfid, ra, dec, ps1OID) in enumerate(zip(df.index, df['raMean'], df['decMean'], df['objID'])): 
        
        if i % 1000 == 0: 
            print(f"i={i}")

        if dec >= 0: dec_int = int(math.floor(dec))
        else: dec_int = int(math.ceil(dec))

        # Open fits file corresponding to declination
        with fits.open(f'../ps1_psc/hlsp_ps1-psc_ps1_gpc1_{dec_int}_multi_v1_cat.fits') as F:
            objid_arr=F[1].data['objid']
            ps_score_arr=F[1].data['ps_score']

            try:
                ps_score = ps_score_arr[np.where(objid_arr == ps1OID)[0][0]]
                #print(ps1OID, ps_score)
            except:
                #print(f"{ps1OID} is not in this file. Set ps_score to -99")
                ps_score = -99
            ps_score_l.append(ps_score)
    return ps_score_l
    
    
# My function: get ps1 object id from transient Ra/Dec
def get_ps1_dr2_objid_df_from_ra_dec(ztfid_l, ra_l, dec_l, df_name, radius=1.0/3600.0):
    if len(ztfid_l)>1:
        df_l = []
        for i, (ztfid, ra, dec) in enumerate(zip(ztfid_l, ra_l, dec_l)):
            
            if i % 1000 == 0: 
                print(f"i={i}: Saving!")
                if i != 0:
                    temp_df = pd.concat(df_l)
                    temp_df.to_csv(f'../dataframes/{df_name}_i={i}.csv') 
            
            ztfid = ztfid
            ra, dec = ra, dec
            radius = radius # radius = 1 arcsec
            constraints = {'nDetections.gt':1}

            # strip blanks and weed out blank and commented-out values
            columns = """objID,raMean,decMean,nDetections""".split(',')
            columns = [x.strip() for x in columns]
            columns = [x for x in columns if x and not x.startswith('#')]

            results = ps1cone(ra,dec,radius,release='dr2',columns=columns,**constraints)
            try:
                tab = ascii.read(results)
                tab = tab.to_pandas()
                tab['ztf_object_id'] = ztfid
                tab = tab.set_index('ztf_object_id')

            except: 
                print(f"Bad results for {ztfid}, ra={ra}, dec={dec}. Add dummy var. Continue!")
                print(results)
                
                temp_df = pd.DataFrame(zip([ztfid], [-99], [ra], [dec], [-99]), columns=['ztf_object_id', 'objID', 'raMean', 'decMean', 'nDetections'])
                tab = temp_df.set_index('ztf_object_id')
                
            df_l.append(tab)
        tab_df = pd.concat(df_l)
        
    else: # One entry
        ztfid = ztfid_l
        ra, dec = ra_l, dec_l
        radius = radius # radius = 1 arcsec
        constraints = {'nDetections.gt':1}

        # strip blanks and weed out blank and commented-out values
        columns = """objID,raMean,decMean,nDetections""".split(',')
        columns = [x.strip() for x in columns]
        columns = [x for x in columns if x and not x.startswith('#')]

        results = ps1cone(ra,dec,radius,release='dr2',columns=columns,**constraints)
        try:
            tab = ascii.read(results)
            tab = tab.to_pandas()
            tab['ztf_object_id'] = ztfid
            tab = tab.set_index('ztf_object_id')
        except: 
            print("Bad results. Add dummy var. Return!")
            print(results)
            
            temp_df = pd.DataFrame(zip([ztfid], [-99], [ra], [dec], [-99]), 
            columns=['ztf_object_id', 'objID', 'raMean', 'decMean', 'nDetections'])
            tab = temp_df.set_index('ztf_object_id')

        tab_df = tab
    return tab_df


def ps1cone(ra,dec,radius,table="mean",release="dr1",format="csv",columns=None,
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,
           **kw):
    """Do a cone search of the PS1 catalog
    
    Parameters
    ----------
    ra (float): (degrees) J2000 Right Ascension
    dec (float): (degrees) J2000 Declination
    radius (float): (degrees) Search radius (<= 0.5 degrees)
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2)
    """
    
    data = kw.copy()
    data['ra'] = ra
    data['dec'] = dec
    data['radius'] = radius
    return ps1search(table=table,release=release,format=format,columns=columns,
                    baseurl=baseurl, verbose=verbose, **data)


def ps1search(table="mean",release="dr1",format="csv",columns=None,
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,
           **kw):
    """Do a general search of the PS1 catalog (possibly without ra/dec/radius)
    
    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2).  Note this is required!
    """
    
    data = kw.copy()
    if not data:
        raise ValueError("You must specify some parameters for search")
    checklegal(table,release)
    if format not in ("csv","votable","json"):
        raise ValueError("Bad value for format")
    url = f"{baseurl}/{release}/{table}.{format}"
    if columns:
        # check that column values are legal
        # create a dictionary to speed this up
        dcols = {}
        for col in ps1metadata(table,release)['name']:
            dcols[col.lower()] = 1
        badcols = []
        for col in columns:
            if col.lower().strip() not in dcols:
                badcols.append(col)
        if badcols:
            raise ValueError('Some columns not found in table: {}'.format(', '.join(badcols)))
        # two different ways to specify a list of column values in the API
        # data['columns'] = columns
        data['columns'] = '[{}]'.format(','.join(columns))

# either get or post works
#    r = requests.post(url, data=data)
    r = requests.get(url, params=data)

    if verbose:
        print(r.url)
    r.raise_for_status()
    if format == "json":
        return r.json()
    else:
        return r.text


def checklegal(table,release):
    """Checks if this combination of table and release is acceptable
    
    Raises a VelueError exception if there is problem
    """
    
    releaselist = ("dr1", "dr2")
    if release not in ("dr1","dr2"):
        raise ValueError("Bad value for release (must be one of {})".format(', '.join(releaselist)))
    if release=="dr1":
        tablelist = ("mean", "stack")
    else:
        tablelist = ("mean", "stack", "detection")
    if table not in tablelist:
        raise ValueError("Bad value for table (for {} must be one of {})".format(release, ", ".join(tablelist)))


def ps1metadata(table="mean",release="dr1",
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs"):
    """Return metadata for the specified catalog and table
    
    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    baseurl: base URL for the request
    
    Returns an astropy table with columns name, type, description
    """
    
    checklegal(table,release)
    url = f"{baseurl}/{release}/{table}/metadata"
    r = requests.get(url)
    r.raise_for_status()
    v = r.json()
    # convert to astropy table
    tab = Table(rows=[(x['name'],x['type'],x['description']) for x in v],
               names=('name','type','description'))
    return tab


def mastQuery(request):
    """Perform a MAST query.

    Parameters
    ----------
    request (dictionary): The MAST request json object

    Returns head,content where head is the response HTTP headers, and content is the returned data
    """
    
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    # Making the query
    conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode('utf-8')

    # Close the https connection
    conn.close()

    return head,content


def resolve(name):
    """Get the RA and Dec for an object using the MAST name resolver
    
    Parameters
    ----------
    name (str): Name of object

    Returns RA, Dec tuple with position"""

    resolverRequest = {'service':'Mast.Name.Lookup',
                       'params':{'input':name,
                                 'format':'json'
                                },
                      }
    headers,resolvedObjectString = mastQuery(resolverRequest)
    resolvedObject = json.loads(resolvedObjectString)
    # The resolver returns a variety of information about the resolved object, 
    # however for our purposes all we need are the RA and Dec
    try:
        objRa = resolvedObject['resolvedCoordinate'][0]['ra']
        objDec = resolvedObject['resolvedCoordinate'][0]['decl']
    except IndexError as e:
        raise ValueError("Unknown object '{}'".format(name))
    return (objRa, objDec)

In [2]:
with fits.open('../ps1_psc/hlsp_ps1-psc_ps1_gpc1_0_multi_v1_cat.fits') as F:
    print(F.info())
    print(F[0].header.cards)
    objid=F[1].data['objid']
    ps_score=F[1].data['ps_score']

Filename: ../ps1_psc/hlsp_ps1-psc_ps1_gpc1_0_multi_v1_cat.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   ()      
  1                1 BinTableHDU     14   16682163R x 2C   [K, E]   
None
('SIMPLE', True, 'conforms to FITS standard')
('BITPIX', 8, 'array data type')
('NAXIS', 0, 'number of array dimensions')
('EXTEND', True, '')
('HLSPTARG', '0', '')
('INSTRUME', 'PS1 survey', '')
('OBSERVAT', 'Haleakala Observatories', '')
('TELESCOP', 'Pan-STARRS1 Telescope', '')
('HLSPID', 'ps1_psc', '')
('HLSPLEAD', 'Adam A Miller', '')
('HLSPNAME', 'Probabilistic Classifications of Unresolved Point Sources in PanSTARRS1', '')
('HLSPVER', 'v1', '')
('LICENSE', 'CC BY 4.0', '')
('LICENURL', 'https://creativecommons.org/licenses/by/4.0/', '')
('REFERENC', 'https://doi.org/10.1088/1538-3873/aae3d9', '')
('HLSPDOI', 'https://doi.org/10.17909/t9-xjrf-7g34', '')


In [3]:
len(objid)

16682163

In [4]:
objid

array([108082954545197165, 108042899536471576, 108173097098982498, ...,
       108862811948172319, 109012792577601116, 108910340111860145])

In [5]:
ps_score[np.where(objid == 108082954545197165)[0][0]]

0.5813869

In [6]:
# score of 0 corresponding to extended objects and 1 corresponding to point sources
# extended objects are resolved galaxies
# point sources are unresolved stars or QSOs

In [7]:
len(ps_score)

16682163

In [8]:
ps_score

array([0.5813869, 0.8385   , 0.99875  , ..., 0.5611875, 0.9922917,
       0.       ], dtype=float32)

In [9]:
%%time

# Instead of above, read in from csv files with host gal info!
# Visually vetted GHOST results split up over XXX files
# So merge them

dfs = []
for start_idx, end_idx in zip(range(0, 90100, 100), range(100, 90200, 100)): #90199, 90299
    df = pd.read_csv(f'./host_info/dataset_bank_hosts_start_idx:end_idx={start_idx}:{end_idx}.csv')
    dfs.append(df)
    
g_hosts_df = pd.concat(dfs, axis=0).reset_index(drop=True)
g_hosts_df.drop_duplicates(subset="TransientName", keep='first', inplace=True)
g_hosts_df = g_hosts_df.set_index('TransientName')
g_hosts_df.sort_values('TransientName', inplace=True)
#g_hosts_df.reset_index(inplace=True, drop=True)
g_hosts_df

CPU times: user 16.9 s, sys: 899 ms, total: 17.8 s
Wall time: 18.3 s


,objName,objAltName1,objAltName2,objAltName3,objID,uniquePspsOBid,ippObjID,surveyID,htmID,zoneID,...,primaryDetection_y,gpetR90,rpetR90,ipetR90,zpetR90,ypetR90,NED_redshift_flag,hasSimbad,class,level_0
TransientName,,,,,,,,,,,,,,,,,,,,,
AT2019ye,PSO J135804.487+203809.056,NaN,NaN,NaN,132762095186853570,916270000024592,143378893266959,0,14931762990582,13276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PS18acw,PSO J203708.962-044523.671,NaN,NaN,NaN,102293092873892523,1049658000060517,397026776902759,0,12246887638605,10229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PS18kz,PSO J132531.079-142122.461,NaN,NaN,NaN,90772013794782874,1040248000047767,422246824852118,0,11265143992289,9077,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SDSS-IISN17461,PSO J034305.548+001507.550,NaN,NaN,NaN,107690557731417905,943938000022176,365716465276576,0,9423182795338,10769,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SN2016ezh,PSO J015804.744+005221.791,NaN,NaN,NaN,106950295197783171,927303000104953,363457312561658,0,8961418540888,10695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21acqetxn,PSO J235011.541+191045.231,NaN,NaN,NaN,131013575480825509,1076778000026511,161254547154832,0,13215295323352,13101,...,1.0,11.96380,5.68306,3.76124,3.02015,NaN,NaN,0.0,NaN,NaN
ZTF21acqnurt,PSO J223925.671-022448.679,NaN,NaN,NaN,105103398569764184,1076249000094978,400995326718721,0,12661889120003,10510,...,1.0,15.28430,2.52730,1.53292,1.24170,2.55906,NaN,0.0,False,NaN
ZTF21acqoloi,PSO J231829.296-011726.224,NaN,NaN,NaN,106453496221281681,1082715000043887,401837140257649,0,12704687240487,10645,...,1.0,NaN,9.90200,2.44196,1.46597,3.27679,NaN,0.0,NaN,NaN


In [10]:
g_hosts_has_ps1_objid_df = g_hosts_df[~g_hosts_df.objID.isna()]
g_hosts_has_ps1_objid_df.index.rename("ztf_object_id", inplace=True)
g_hosts_has_ps1_objid_df = g_hosts_has_ps1_objid_df[g_hosts_has_ps1_objid_df.index.str.startswith('ZTF')]
g_hosts_has_ps1_objid_df = g_hosts_has_ps1_objid_df[g_hosts_has_ps1_objid_df.index.str.contains("\n")==False]
g_hosts_has_ps1_objid_df.to_csv('../dataframes/g_hosts_has_ps1_objid_df.csv')
g_hosts_has_ps1_objid_df

,objName,objAltName1,objAltName2,objAltName3,objID,uniquePspsOBid,ippObjID,surveyID,htmID,zoneID,...,primaryDetection_y,gpetR90,rpetR90,ipetR90,zpetR90,ypetR90,NED_redshift_flag,hasSimbad,class,level_0
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaaenu,PSO J014921.296+353104.780,NaN,NaN,NaN,150620273387282157,942081000036636,201335181971228,0,17342042780099,15062,...,1.0,13.51850,4.98918,4.267330,3.78907,9.400590,NaN,0.0,False,NaN
ZTF17aaaazhi,PSO J021351.579+333011.621,NaN,NaN,NaN,148200334649204503,936092000041298,201897822691666,0,17567891492385,14820,...,1.0,5.74010,1.27822,0.996905,1.49481,0.999151,NaN,0.0,False,NaN
ZTF17aaaazmz,PSO J020108.703+354438.836,NaN,NaN,NaN,150890302862623476,944990000044342,201356656815413,0,17342424965577,15089,...,1.0,4.73348,2.72390,3.379060,3.34216,4.355060,NaN,0.0,False,NaN
ZTF17aaabghb,PSO J050857.900-010153.780,NaN,NaN,NaN,106760772412722499,969245000054665,368151711765898,0,9372013840704,10676,...,1.0,5.70637,5.27978,7.873000,3.75759,2.645650,NaN,0.0,False,NaN
ZTF17aaabuab,PSO J032818.391-021919.410,NaN,NaN,NaN,105210520766363928,947850000033169,365261198754193,0,9429305171894,10521,...,1.0,3.08845,4.04116,4.027430,3.76517,11.518300,NaN,0.0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21acqetxn,PSO J235011.541+191045.231,NaN,NaN,NaN,131013575480825509,1076778000026511,161254547154832,0,13215295323352,13101,...,1.0,11.96380,5.68306,3.761240,3.02015,NaN,NaN,0.0,NaN,NaN
ZTF21acqnurt,PSO J223925.671-022448.679,NaN,NaN,NaN,105103398569764184,1076249000094978,400995326718721,0,12661889120003,10510,...,1.0,15.28430,2.52730,1.532920,1.24170,2.559060,NaN,0.0,False,NaN
ZTF21acqoloi,PSO J231829.296-011726.224,NaN,NaN,NaN,106453496221281681,1082715000043887,401837140257649,0,12704687240487,10645,...,1.0,NaN,9.90200,2.441960,1.46597,3.276790,NaN,0.0,NaN,NaN


# TRANSIENT (Not Host) ps1 objIDs

In [11]:
%%time
transient_df_name = 'get_ps1_dr2_objid_transient_coords_df'
hosts_df_name = 'get_ps1_dr2_objid_ghost_hosts_assoc_coords_df'

get_ps1_dr2_objid_transient_coords_df = get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=g_hosts_has_ps1_objid_df.index, 
                                 ra_l=g_hosts_has_ps1_objid_df['TransientRA'],
                                 dec_l=g_hosts_has_ps1_objid_df['TransientDEC'],
                                 df_name=transient_df_name)
get_ps1_dr2_objid_transient_coords_df.to_csv(f'../dataframes/{transient_df_name}.csv')
get_ps1_dr2_objid_transient_coords_df

i=0: Saving!
Bad results for ZTF17aabcirl, ra=19.51308273339673, dec=23.40903684468689. Add dummy var. Continue!

Bad results for ZTF17aabfgmo, ra=54.5358249568194, dec=-1.5676953136407263. Add dummy var. Continue!

Bad results for ZTF17aabvisz, ra=69.0042197, dec=-25.967403. Add dummy var. Continue!

Bad results for ZTF17aadlcod, ra=19.04491174207884, dec=5.397605826541337. Add dummy var. Continue!

Bad results for ZTF17aadnacr, ra=36.751739677434415, dec=-11.087388745230058. Add dummy var. Continue!

Bad results for ZTF18aaacqmh, ra=149.52551498359415, dec=10.359362928128135. Add dummy var. Continue!

Bad results for ZTF18aaaglmc, ra=140.96258132267957, dec=75.8146859768065. Add dummy var. Continue!

Bad results for ZTF18aaavrep, ra=178.88965132264923, dec=35.7434854250769. Add dummy var. Continue!

Bad results for ZTF18aabdnbr, ra=173.0291548499705, dec=47.10102615002268. Add dummy var. Continue!

Bad results for ZTF18aabeszt, ra=221.212894757904, dec=43.3167923710718. Add dummy var

Bad results for ZTF18aauikmq, ra=185.6985180817963, dec=27.19881785463832. Add dummy var. Continue!

Bad results for ZTF18aautopz, ra=205.97228601958128, dec=61.55470548263715. Add dummy var. Continue!

Bad results for ZTF18aauzldt, ra=235.87897729999835, dec=43.23614712858126. Add dummy var. Continue!

Bad results for ZTF18aavkszo, ra=275.20012360024714, dec=53.65025072042829. Add dummy var. Continue!

Bad results for ZTF18aavlrqz, ra=270.09796256091835, dec=55.58336929261747. Add dummy var. Continue!

Bad results for ZTF18aavpffg, ra=236.95554643503527, dec=29.27037384697612. Add dummy var. Continue!

Bad results for ZTF18aavpfwy, ra=224.38021347273013, dec=36.950705200029006. Add dummy var. Continue!

Bad results for ZTF18aavslaf, ra=237.6907238000124, dec=15.212158557186726. Add dummy var. Continue!

Bad results for ZTF18aawfofp, ra=238.7581687727084, dec=41.57805344092623. Add dummy var. Continue!

Bad results for ZTF18aawjhmu, ra=240.7843676499499, dec=54.97622255756603. Add dumm

Bad results for ZTF18abnysyy, ra=350.62311253199994, dec=37.232130976. Add dummy var. Continue!

Bad results for ZTF18abnzney, ra=0.1867672545454545, dec=15.499429372727274. Add dummy var. Continue!

Bad results for ZTF18abolrnr, ra=7.194448892307692, dec=0.3643056692307692. Add dummy var. Continue!

Bad results for ZTF18aboltfa, ra=1.690094658343254, dec=11.008761208339394. Add dummy var. Continue!

Bad results for ZTF18aboneii, ra=34.01234072728991, dec=-6.544211018195185. Add dummy var. Continue!

Bad results for ZTF18aborbiu, ra=323.6390236166667, dec=8.200699716666668. Add dummy var. Continue!

Bad results for ZTF18abosngi, ra=87.94197484763322, dec=76.09151417375293. Add dummy var. Continue!

Bad results for ZTF18abotdrt, ra=21.780508788459223, dec=-5.0796382038549694. Add dummy var. Continue!

Bad results for ZTF18abpovmh, ra=21.02628511428604, dec=-15.2386871500037. Add dummy var. Continue!

Bad results for ZTF18abpucqe, ra=246.4157962857143, dec=72.0819877142857. Add dummy var

Bad results for ZTF18abvjyki, ra=340.31344004864513, dec=23.412637200066484. Add dummy var. Continue!

Bad results for ZTF18abvkdhm, ra=15.687480446293891, dec=-21.957371240628007. Add dummy var. Continue!

Bad results for ZTF18abvkffr, ra=11.682533015591355, dec=3.741780094011716. Add dummy var. Continue!

Bad results for ZTF18abvkhul, ra=20.340921329168424, dec=30.718573770845303. Add dummy var. Continue!

Bad results for ZTF18abvkmgw, ra=9.361961488886465, dec=15.014233555556643. Add dummy var. Continue!

Bad results for ZTF18abvkugj, ra=41.85885252222732, dec=30.64632297778804. Add dummy var. Continue!

Bad results for ZTF18abvldyw, ra=39.76498352303195, dec=-11.41663773733478. Add dummy var. Continue!

Bad results for ZTF18abvmlow, ra=27.794379618918995, dec=-3.4902496972980543. Add dummy var. Continue!

Bad results for ZTF18abvpkdw, ra=343.5559018202064, dec=-2.817165540994651. Add dummy var. Continue!

Bad results for ZTF18abvrxmh, ra=26.42942431869016, dec=-1.9500873065096729. 

Bad results for ZTF18accnhoj, ra=159.7661914098909, dec=26.65483050010029. Add dummy var. Continue!

Bad results for ZTF18accpjbe, ra=328.690793115428, dec=15.166439243409314. Add dummy var. Continue!

i=6000: Saving!
Bad results for ZTF18accrorf, ra=17.162971992308602, dec=20.07990491538834. Add dummy var. Continue!

Bad results for ZTF18accupqk, ra=122.65158877499768, dec=47.8995469250117. Add dummy var. Continue!

Bad results for ZTF18accvhqz, ra=149.46769295004876, dec=33.15462567324282. Add dummy var. Continue!

Bad results for ZTF18accvkom, ra=188.09079674995, dec=41.80942770006104. Add dummy var. Continue!

Bad results for ZTF18accvkpt, ra=175.15377967142769, dec=42.605993257152505. Add dummy var. Continue!

Bad results for ZTF18accvqmj, ra=135.07984988333337, dec=61.33758747222222. Add dummy var. Continue!

Bad results for ZTF18accwild, ra=157.1360216999828, dec=23.57777257143381. Add dummy var. Continue!

Bad results for ZTF18accwrom, ra=147.33792610001663, dec=24.404601544502

...

Bad results for ZTF21abfmbix, ra=193.8217386, dec=2.897310784615428. Add dummy var. Continue!

Bad results for ZTF21abfmnms, ra=189.38565413749564, dec=25.34274572500459. Add dummy var. Continue!

Bad results for ZTF21abfmuun, ra=210.1492841750017, dec=17.347591175003856. Add dummy var. Continue!

Bad results for ZTF21abfnfdw, ra=214.8149557536005, dec=26.300391985071712. Add dummy var. Continue!

Bad results for ZTF21abfnjya, ra=228.05791024434728, dec=16.850468173370274. Add dummy var. Continue!

Bad results for ZTF21abfokxd, ra=243.16390589999952, dec=14.119780920004928. Add dummy var. Continue!

Bad results for ZTF21abfpgph, ra=239.2424025437545, dec=24.55343496875884. Add dummy var. Continue!

Bad results for ZTF21abfspnh, ra=179.28486065385673, dec=44.17050646925157. Add dummy var. Continue!

Bad results for ZTF21abfxhlw, ra=358.43980909999993, dec=9.242279100000005. Add dummy var. Continue!

Bad results for ZTF21abfxjik, ra=347.4358474412711, dec=-7.576462238644865. Add dummy va

Bad results for ZTF21abkaoxe, ra=249.6685384259248, dec=32.54848963385225. Add dummy var. Continue!

Bad results for ZTF21abkapuy, ra=248.27780203620625, dec=57.383379355923886. Add dummy var. Continue!

Bad results for ZTF21abkavmp, ra=235.86361308863096, dec=7.906820473586748. Add dummy var. Continue!

Bad results for ZTF21abkdtdx, ra=256.5302621221221, dec=68.72287292047227. Add dummy var. Continue!

Bad results for ZTF21abkfbyi, ra=8.349872899094397, dec=6.284675932654835. Add dummy var. Continue!

Bad results for ZTF21abkftes, ra=9.9880815, dec=-0.8885446999999971. Add dummy var. Continue!

Bad results for ZTF21abkkuja, ra=331.62707856386186, dec=9.722919381842075. Add dummy var. Continue!

Bad results for ZTF21abkkwie, ra=356.192607, dec=25.90863946728004. Add dummy var. Continue!

Bad results for ZTF21abklcjq, ra=257.4697384788513, dec=41.22212811367476. Add dummy var. Continue!

Bad results for ZTF21abklnfg, ra=356.93936929656945, dec=-8.386022909257136. Add dummy var. Continue

Bad results for ZTF21abqqcna, ra=31.42773505770285, dec=9.450939122788313. Add dummy var. Continue!

Bad results for ZTF21abqqskc, ra=5.187028896350371, dec=10.59607673727129. Add dummy var. Continue!

Bad results for ZTF21abqsxgo, ra=337.8776523, dec=12.854544884143024. Add dummy var. Continue!

Bad results for ZTF21abqsyfk, ra=328.74393777935273, dec=0.2671430002694706. Add dummy var. Continue!

Bad results for ZTF21abqvwzb, ra=343.86311677242963, dec=4.079264712517542. Add dummy var. Continue!

Bad results for ZTF21abqvzbi, ra=44.43163677740797, dec=6.572217970966904. Add dummy var. Continue!

Bad results for ZTF21abqvzng, ra=45.96475725372416, dec=13.541956073386928. Add dummy var. Continue!

Bad results for ZTF21abqwclr, ra=5.9056904675368544, dec=8.550855849515372. Add dummy var. Continue!

Bad results for ZTF21abrahvj, ra=324.4452477, dec=1.8559720865737568. Add dummy var. Continue!

Bad results for ZTF21abrgbex, ra=245.29379821941032, dec=36.06121786324474. Add dummy var. Conti

Bad results for ZTF21abxdrhx, ra=26.38983593374164, dec=-3.0748285464758. Add dummy var. Continue!

Bad results for ZTF21abxdrsx, ra=37.99757607056221, dec=-26.05075372399856. Add dummy var. Continue!

Bad results for ZTF21abxeaiq, ra=72.73024023567429, dec=1.2534479132824288. Add dummy var. Continue!

Bad results for ZTF21abxfdve, ra=44.4434324, dec=-5.65369170000001. Add dummy var. Continue!

Bad results for ZTF21abxhnep, ra=247.89732870189056, dec=18.78156855489101. Add dummy var. Continue!

Bad results for ZTF21abxmjqd, ra=28.965265752030994, dec=16.399954990540877. Add dummy var. Continue!

Bad results for ZTF21abxmxut, ra=69.79878955569187, dec=-3.9150016751753896. Add dummy var. Continue!

Bad results for ZTF21abxmyfb, ra=74.00819999999997, dec=0.8432368629520823. Add dummy var. Continue!

Bad results for ZTF21abxnghi, ra=109.30841396059616, dec=43.68404631456623. Add dummy var. Continue!

Bad results for ZTF21abxnhpn, ra=47.59393828967369, dec=-3.02061810417095. Add dummy var. 

,objID,raMean,decMean,nDetections
ztf_object_id,,,,
ZTF17aaaaenu,150610273519657838,27.351981,35.514352,61
ZTF17aaaazhi,148190334676288060,33.467597,33.497860,64
ZTF17aaaazmz,150890302783704888,30.278370,35.745211,57
ZTF17aaabghb,106760772362408457,77.236240,-1.026622,77
ZTF17aaabuab,105220520679517720,52.068000,-2.310620,77
...,...,...,...,...
ZTF21acqetxn,131013575639463501,357.563938,19.177452,56
ZTF21acqnurt,105083398645596687,339.864626,-2.428137,87
ZTF21acqoloi,106453496251661481,349.625177,-1.290811,65


# TODO: HOST (not transient) ps1 objIDs

In [12]:
%%time
get_ps1_dr2_objid_ghost_hosts_assoc_coords_df = get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=g_hosts_has_ps1_objid_df.index, 
                                 ra_l=g_hosts_has_ps1_objid_df['raMean'],
                                 dec_l=g_hosts_has_ps1_objid_df['decMean'],
                                 df_name=hosts_df_name)
get_ps1_dr2_objid_ghost_hosts_assoc_coords_df.to_csv(f'../dataframes/{hosts_df_name}.csv')
get_ps1_dr2_objid_ghost_hosts_assoc_coords_df

i=0: Saving!
i=1000: Saving!
i=2000: Saving!
Bad results for ZTF18abgucur, ra=12.46594707, dec=39.52371288. Add dummy var. Continue!

i=3000: Saving!
Bad results for ZTF18abrjxdh, ra=242.32539598, dec=75.18739901. Add dummy var. Continue!

i=4000: Saving!
i=5000: Saving!
i=6000: Saving!
i=7000: Saving!
i=8000: Saving!
i=9000: Saving!
Bad results for ZTF19aaapnnx, ra=173.92004286, dec=-10.30142226. Add dummy var. Continue!

i=10000: Saving!
i=11000: Saving!
i=12000: Saving!
i=13000: Saving!
Bad results for ZTF19ackxpgy, ra=12.46594707, dec=39.52371288. Add dummy var. Continue!

i=14000: Saving!
i=15000: Saving!
i=16000: Saving!
i=17000: Saving!
i=18000: Saving!
i=19000: Saving!
Bad results for ZTF20acxfhha, ra=224.24836255, dec=-4.82745548. Add dummy var. Continue!

i=20000: Saving!
i=21000: Saving!
i=22000: Saving!
i=23000: Saving!
CPU times: user 29min 20s, sys: 2min 19s, total: 31min 40s
Wall time: 4h 15min 15s


,objID,raMean,decMean,nDetections
ztf_object_id,,,,
ZTF17aaaaenu,150620273387282157,27.338759,35.517955,38
ZTF17aaaazhi,148200334649204503,33.464886,33.503208,40
ZTF17aaaazmz,150890302862623476,30.286253,35.744116,55
ZTF17aaaazmz,150890302862863079,30.286215,35.744006,3
ZTF17aaabghb,106760772412722499,77.241261,-1.031607,54
...,...,...,...,...
ZTF21acqnurt,105103398569764184,339.857006,-2.413547,46
ZTF21acqoloi,106453496221281681,349.622099,-1.290629,44
ZTF21acqqnqg,92013304110782233,330.411182,-13.323487,3


# Get PS1_PSC scores, make cuts

In [17]:
%%time
ps_score_transient_coords_l = get_psc_score(df=get_ps1_dr2_objid_transient_coords_df)

get_ps1_dr2_objid_transient_coords_df['pgal_transient'] = ps_score_transient_coords_l
get_ps1_dr2_objid_transient_coords_df.to_csv(f'../dataframes/{transient_df_name}_pgal_transient.csv')
get_ps1_dr2_objid_transient_coords_df

i=0
i=1000
i=2000
i=3000
i=4000
i=5000
i=6000
i=7000
i=8000
i=9000
i=10000
i=11000
i=12000
i=13000
i=14000
i=15000
i=16000
i=17000
i=18000
i=19000
i=20000
i=21000
i=22000
i=23000
i=24000
CPU times: user 26min 53s, sys: 42min 5s, total: 1h 8min 59s
Wall time: 4h 31min 38s


,objID,raMean,decMean,nDetections,pgal_transient
ztf_object_id,,,,,
ZTF17aaaaenu,150610273519657838,27.351981,35.514352,61,0.985625
ZTF17aaaazhi,148190334676288060,33.467597,33.497860,64,0.999167
ZTF17aaaazmz,150890302783704888,30.278370,35.745211,57,0.990000
ZTF17aaabghb,106760772362408457,77.236240,-1.026622,77,0.990250
ZTF17aaabuab,105220520679517720,52.068000,-2.310620,77,1.000000
...,...,...,...,...,...
ZTF21acqetxn,131013575639463501,357.563938,19.177452,56,0.993333
ZTF21acqnurt,105083398645596687,339.864626,-2.428137,87,0.996250
ZTF21acqoloi,106453496251661481,349.625177,-1.290811,65,0.987083


In [18]:
%%time
ps_score_ghost_hosts_assoc_coords_l = get_psc_score(df=get_ps1_dr2_objid_ghost_hosts_assoc_coords_df)

get_ps1_dr2_objid_ghost_hosts_assoc_coords_df['pgal_host'] = ps_score_ghost_hosts_assoc_coords_l
get_ps1_dr2_objid_ghost_hosts_assoc_coords_df.to_csv(f'../dataframes/{hosts_df_name}_pgal_host.csv')
get_ps1_dr2_objid_ghost_hosts_assoc_coords_df

i=0
i=1000
i=2000
i=3000
i=4000
i=5000
i=6000
i=7000
i=8000
i=9000
i=10000
i=11000
i=12000
i=13000
i=14000
i=15000
i=16000
i=17000
i=18000
i=19000
i=20000
i=21000
i=22000
i=23000
i=24000
i=25000
i=26000
i=27000
CPU times: user 29min 37s, sys: 44min 5s, total: 1h 13min 43s
Wall time: 4h 41min 5s


,objID,raMean,decMean,nDetections,pgal_host
ztf_object_id,,,,,
ZTF17aaaaenu,150620273387282157,27.338759,35.517955,38,0.025833
ZTF17aaaazhi,148200334649204503,33.464886,33.503208,40,0.929583
ZTF17aaaazmz,150890302862623476,30.286253,35.744116,55,0.000000
ZTF17aaaazmz,150890302862863079,30.286215,35.744006,3,-99.000000
ZTF17aaabghb,106760772412722499,77.241261,-1.031607,54,0.003750
...,...,...,...,...,...
ZTF21acqnurt,105103398569764184,339.857006,-2.413547,46,0.052833
ZTF21acqoloi,106453496221281681,349.622099,-1.290629,44,0.001667
ZTF21acqqnqg,92013304110782233,330.411182,-13.323487,3,-99.000000


In [21]:
# Make keep_df
# x is host, y is transient
keep_df = pd.merge(get_ps1_dr2_objid_ghost_hosts_assoc_coords_df, get_ps1_dr2_objid_transient_coords_df, left_index=True, right_index=True)
keep_df

,objID_x,raMean_x,decMean_x,nDetections_x,pgal_host,objID_y,raMean_y,decMean_y,nDetections_y,pgal_transient
ztf_object_id,,,,,,,,,,
ZTF17aaaaenu,150620273387282157,27.338759,35.517955,38,0.025833,150610273519657838,27.351981,35.514352,61,0.985625
ZTF17aaaazhi,148200334649204503,33.464886,33.503208,40,0.929583,148190334676288060,33.467597,33.497860,64,0.999167
ZTF17aaaazmz,150890302862623476,30.286253,35.744116,55,0.000000,150890302783704888,30.278370,35.745211,57,0.990000
ZTF17aaaazmz,150890302862863079,30.286215,35.744006,3,-99.000000,150890302783704888,30.278370,35.745211,57,0.990000
ZTF17aaabghb,106760772412722499,77.241261,-1.031607,54,0.003750,106760772362408457,77.236240,-1.026622,77,0.990250
...,...,...,...,...,...,...,...,...,...,...
ZTF21acqnurt,105103398569764184,339.857006,-2.413547,46,0.052833,105083398645596687,339.864626,-2.428137,87,0.996250
ZTF21acqoloi,106453496221281681,349.622099,-1.290629,44,0.001667,106453496251661481,349.625177,-1.290811,65,0.987083
ZTF21acqqnqg,92013304110782233,330.411182,-13.323487,3,-99.000000,92013304111661908,330.411215,-13.323722,124,-99.000000


In [22]:
keep_df = keep_df[(keep_df.pgal_host < 0.5) & (keep_df.pgal_host != -99) & (keep_df.pgal_transient < 0.5)]
keep_df.to_csv('../dataframes/keep_df.csv')
keep_df

,objID_x,raMean_x,decMean_x,nDetections_x,pgal_host,objID_y,raMean_y,decMean_y,nDetections_y,pgal_transient
ztf_object_id,,,,,,,,,,
ZTF17aaadvfh,156720101231743602,10.123150,40.602459,52,0.079583,156720101231743602,10.123150,40.602459,52,0.079583
ZTF17aaahrni,102970804180328356,80.418019,-4.184979,46,0.004167,102970804232846942,80.423322,-4.186241,4,0.322083
ZTF17aaaiujz,167221141795271735,114.179554,49.350772,24,0.002500,167211141963355377,114.196321,49.345549,93,-99.000000
ZTF17aaatuld,152420291435071321,29.143479,37.017250,63,0.013958,152410291469574894,29.147008,37.011882,64,-99.000000
ZTF17aaatuld,152420291435071321,29.143479,37.017250,63,0.013958,152410291470514558,29.146982,37.011638,3,-99.000000
...,...,...,...,...,...,...,...,...,...,...
ZTF21aclbipj,152310245923140842,24.592315,36.925186,23,0.354042,152310245923140842,24.592315,36.925186,23,0.354042
ZTF21acldmwy,86950870276547146,87.027691,-17.536090,59,0.010458,86950870276547146,87.027691,-17.536090,59,0.010458
ZTF21acngnza,158441551680351445,155.168183,42.034115,63,0.000000,158441551680351445,155.168183,42.034115,63,0.000000


In [ ]:
# These ZTF18aalsfbd, ZTF18aalroha, ZTF18aaacqmh are transients at start of ztf survey...
# Others like ZTF17aabcirl, ZTF17aabfgmo, ZTF17aabvisz, ZTF17aadnacr are stars...

In [13]:
get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=['ZTF17aaaessf'], ra_l=6.99479767152418, dec_l=34.47048854338335, df_name="blah", radius=9.0/3600.0)

,objID,raMean,decMean,nDetections
ztf_object_id,,,,
ZTF17aaaessf,149360069947635231,6.994798,34.470489,59


In [51]:
get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=['ZTF18aalroha'], ra_l=217.0604874344824, dec_l=40.866147144851006, radius=2.0/3600.0)

Bad results. See output. Then skip!



In [53]:
get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=['ZTF17aabcirl'], ra_l=19.51308273339673, dec_l=23.40903684468689, radius=1.50/3600.0)

Bad results. See output. Then skip!



In [34]:
df_ah = pd.read_csv(f'../dataframes/get_ps1_dr2_objid_transient_coords_df_i=200.csv')
df_ah

,ztf_object_id,objID,raMean,decMean,nDetections
0,ZTF17aaaaenu,150610273519657838,27.351981,35.514352,61
1,ZTF17aaaazhi,148190334676288060,33.467597,33.497860,64
2,ZTF17aaaazmz,150890302783704888,30.278370,35.745211,57
3,ZTF17aaabghb,106760772362408457,77.236240,-1.026622,77
4,ZTF17aaabuab,105220520679517720,52.068000,-2.310620,77
...,...,...,...,...,...
207,ZTF17aabqajc,157260061006557656,6.100616,41.055845,63
208,ZTF17aabrisn,113130319147436034,31.914750,4.279631,70
209,ZTF17aabsamc,96890801274185678,80.127441,-9.253931,85
210,ZTF17aabslfb,163651087504864995,108.750441,46.378624,2


In [20]:
test2 = g_hosts_has_ps1_objid_df
test2 = test2[test2.index.str.contains("\n")==False]
g_hosts_has_ps1_objid_df.iloc[19:25]

,objName,objAltName1,objAltName2,objAltName3,objID,uniquePspsOBid,ippObjID,surveyID,htmID,zoneID,...,primaryDetection_y,gpetR90,rpetR90,ipetR90,zpetR90,ypetR90,NED_redshift_flag,hasSimbad,class,level_0
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaessf,PSO J002758.152+342737.379,NaN,NaN,NaN,149350069924292964,910743000025002,198689482105259,0,17242199554318,14935,...,1.0,8.50073,3.59930,1.34110,1.30147,6.296310,NaN,0.0,False,NaN
ZTF17aaaessf\nZTF17aaaessf,PSO J002758.152+342737.379,NaN,NaN,NaN,149350069924292964,910743000025002,198689482105259,0,17242199554318,14935,...,1.0,8.50073,3.59930,1.34110,1.30147,6.296310,NaN,0.0,False,NaN
ZTF17aaaeszu,PSO J002725.144+410817.821,NaN,NaN,NaN,157360068547966611,910400000060829,234397840240029,0,17318542320870,15736,...,1.0,8.56861,3.88292,5.23500,4.23818,3.732670,NaN,0.0,False,NaN
ZTF17aaafssi,PSO J012811.769+304923.587,NaN,NaN,NaN,144980220490828453,924746000021116,200489073398395,0,17348710249472,14498,...,1.0,11.74970,3.79276,1.34208,1.21408,1.012490,NaN,0.0,False,NaN
ZTF17aaafstx,PSO J012907.611+303613.108,NaN,NaN,NaN,144720222814224929,924746000062710,200489073439989,0,17348703370033,14472,...,1.0,7.89906,3.50652,2.62227,3.55783,NaN,NaN,0.0,False,NaN
ZTF17aaafvft,PSO J021026.532-101523.141,NaN,NaN,NaN,95690326106512679,930138000016982,404238026949205,0,8992209949747,9569,...,1.0,7.73635,1.79648,3.03877,1.37975,0.611368,NaN,0.0,False,NaN


# TODO: match PS1 object to transient ra/dec, then apply T&M ps_score

In [11]:
#g_hosts_has_ps1_objid_df = g_hosts_has_ps1_objid_df.head()

In [12]:
g_hosts_has_ps1_objid_df

,objName,objAltName1,objAltName2,objAltName3,objID,uniquePspsOBid,ippObjID,surveyID,htmID,zoneID,...,primaryDetection_y,gpetR90,rpetR90,ipetR90,zpetR90,ypetR90,NED_redshift_flag,hasSimbad,class,level_0
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaaenu,PSO J014921.296+353104.780,NaN,NaN,NaN,150620273387282157,942081000036636,201335181971228,0,17342042780099,15062,...,1.0,13.51850,4.98918,4.267330,3.78907,9.400590,NaN,0.0,False,NaN
ZTF17aaaazhi,PSO J021351.579+333011.621,NaN,NaN,NaN,148200334649204503,936092000041298,201897822691666,0,17567891492385,14820,...,1.0,5.74010,1.27822,0.996905,1.49481,0.999151,NaN,0.0,False,NaN
ZTF17aaaazmz,PSO J020108.703+354438.836,NaN,NaN,NaN,150890302862623476,944990000044342,201356656815413,0,17342424965577,15089,...,1.0,4.73348,2.72390,3.379060,3.34216,4.355060,NaN,0.0,False,NaN
ZTF17aaabghb,PSO J050857.900-010153.780,NaN,NaN,NaN,106760772412722499,969245000054665,368151711765898,0,9372013840704,10676,...,1.0,5.70637,5.27978,7.873000,3.75759,2.645650,NaN,0.0,False,NaN
ZTF17aaabuab,PSO J032818.391-021919.410,NaN,NaN,NaN,105210520766363928,947850000033169,365261198754193,0,9429305171894,10521,...,1.0,3.08845,4.04116,4.027430,3.76517,11.518300,NaN,0.0,False,NaN


In [13]:
%%time
for ztfid, ra, dec, ps1OID in zip(g_hosts_has_ps1_objid_df.index,
                           g_hosts_has_ps1_objid_df['raMean'], 
                           g_hosts_has_ps1_objid_df['decMean'], 
                           g_hosts_has_ps1_objid_df['objID']):    
    
    
    if dec >= 0: dec_int = int(math.floor(dec))
    else: dec_int = int(math.ceil(dec))
    
    # Open fits file corresponding to declination
    with fits.open(f'../ps1_psc/hlsp_ps1-psc_ps1_gpc1_{dec_int}_multi_v1_cat.fits') as F:
        objid_arr=F[1].data['objid']
        ps_score_arr=F[1].data['ps_score']
        
        try:
            ps_score = ps_score_arr[np.where(objid_arr == ps1OID)[0][0]]
            print(ps1OID, ps_score)
        except:
            print(f"{ps1OID} is not in this file. Set ps_score to -99")
            ps_score = -99

150620273387282157 0.025833333
148200334649204503 0.9295833
150890302862623476 0.0
106760772412722499 0.00375
105210520766363928 0.016416667
CPU times: user 366 ms, sys: 563 ms, total: 929 ms
Wall time: 3.67 s


In [15]:
for ztfid, (ra_t, dec_t, ra_host, dec_host) in g_hosts_has_ps1_objid_df[['TransientRA', 'TransientDEC', 'raMean', 'decMean']].iterrows():
    print(ztfid)
    print(f'Transient: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos={ra_t}+{dec_t}&filter=color')
    #ps1_dr2_objid = get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=[ztfid], ra_l=ra_t, dec_l=dec_t)
    #print(ps1_dr2_objid)
    print(f'Host: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos={ra_host}+{dec_host}&filter=color')
    #ps1_dr2_objid = get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=[ztfid], ra_l=ra_host, dec_l=dec_host)
    #print(ps1_dr2_objid)
    print(f'\n')


ZTF17aaaaenu
Transient: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=27.35194614323238+35.51434387930057&filter=color
                            objID     raMean    decMean  nDetections
ztf_object_id                                                       
ZTF17aaaaenu   150610273519657838  27.351981  35.514352           61
Host: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=27.33876949+35.5179556&filter=color
                            objID     raMean    decMean  nDetections
ztf_object_id                                                       
ZTF17aaaaenu   150620273387282157  27.338759  35.517955           38


ZTF17aaaazhi
Transient: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=33.467575772476465+33.4979008400576&filter=color
                            objID     raMean   decMean  nDetections
ztf_object_id                                                      
ZTF17aaaazhi   148190334676288060  33.467597  33.49786           64
Host: http://ps1images.stsci.edu/cgi-bin/p

ValueError: Length of values (1) does not match length of index (2)

In [16]:
get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=['ZTF17aaaaenu'], ra_l=27.33876949, dec_l=35.5179556)

,objID,raMean,decMean,nDetections
ztf_object_id,,,,
ZTF17aaaaenu,150620273387282157,27.338759,35.517955,38


In [18]:
get_ps1_dr2_objid_df = get_ps1_dr2_objid_df_from_ra_dec(ztfid_l=g_hosts_has_ps1_objid_df.index, 
                                 ra_l=g_hosts_has_ps1_objid_df['TransientRA'],
                                 dec_l=g_hosts_has_ps1_objid_df['TransientDEC'])
get_ps1_dr2_objid_df.to_csv('../dataframes/get_ps1_dr2_objid_df.csv')
get_ps1_dr2_objid_df

,objID,raMean,decMean,nDetections
ztf_object_id,,,,
ZTF17aaaaenu,150610273519657838,27.351981,35.514352,61
ZTF17aaaazhi,148190334676288060,33.467597,33.497860,64
ZTF17aaaazmz,150890302783704888,30.278370,35.745211,57
ZTF17aaabghb,106760772362408457,77.236240,-1.026622,77
ZTF17aaabuab,105220520679517720,52.068000,-2.310620,77
